<a href="https://colab.research.google.com/github/GmoWilliams/StackingBoxAgents/blob/main/StackRobot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Multiagentes de acomodo de cajas 

#### *Elaborado por:* 
#### - Jesús Enríquez Jaime
#### - Valter Kuhne
#### - José Emilio Flores Figueroa
#### - Guillermo Williams

[Repositorio en Github](https://github.com/GmoWilliams/StackingBoxAgents)

#####*Primero instalamos la herramienta de agentpy*

In [1]:
!pip install agentpy

#####*Importamos las dependencias necesarias para las operaciones*

In [2]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
from random import randrange, uniform
from matplotlib.animation import FuncAnimation

#####*Definimos la clase para el agente de nuestra categoría: Robot (Los Roombas)*
######**Setup**
######Este objeto se moverá en espacios tipo "grid" de la biblioteca agentpy
######Le asignamos el color 0 (gris)
######Le asignamos el status 0 (el robot no tiene actualmente una caja)
######**Find New Cell**
######Definimos su función para moverse únicamente un espacio en un eje de ocho direcciones
######Si el robot no carga una caja actualmente (status 0), se mueve libremente dentro del grid en búsqueda de cajas
######Si el robot carga con una caja (status 1), se mueve hasta el limite izquierdo de la pantalla y después hacia el limite superior con cada paso, su color cambia a verde (typeColor 3) en señal de que carga una caja

In [3]:
class StackBot(ap.Agent):

    def setup(self):
        """ Initiate agent attributes. """
        self.grid = self.model.grid
        self.random = self.model.random
        self.typeColor = 0
        self.status = 0
        self.box = None

    def find_new_cell(self, stockSpot, posr, num_moves):

      if self.status == 0:
        """ Move to random free spot and update free spots. """
        num1 = 0
        num2 = 0
        while (num1 == 0 and num2 == 0 ):
          num1 = randrange(-1, 2)
          num2 = randrange(-1, 2)
        self.grid.move_by(self, (num1, num2))
        self.typeColor = 0

      elif self.status == 1:
        """ Take box into spot position to stack them """
        x = 0
        y = 0
        
        # Checks if the robot is not already at the left
        if (posr[0] > 0):
          x = -1
        # Checks if the robot is not already at the top
        if (posr[1] > stockSpot):
          y = -1
        elif (posr[1] < stockSpot):
          y = 1
        
        self.grid.move_by(self, (x, y))
        self.typeColor = 3
        # print("------------\nStockSpot is: ( 0 ,",stockSpot,"), \nEl robot ",self.id," está en la posición: ",posr,"\nEste es el paso número: ",num_moves,"\n------------")

#####*Definimos la clase para el agente de nuestra segunda categoría: Cell (Las Cajas)*
######**Setup**
######Se inicializan sus valores del mismo modo pero agregando un parámetro para verificar si la caja se ha recogido (status 0), y un color naranja (typeColor 1)

In [4]:
class Cell(ap.Agent):

    def setup(self):
        """ Initiate agent attributes. """
        self.grid = self.model.grid
        self.random = self.model.random
        self.stacked = False
        self.typeColor = 1
        self.status = 0

    def find_new_cell(self, posr):
        if self.status == 1:
          x = 0
          y = 0
          
          # Checks if the robot is not already at the left
          if (posr[0] > 0):
            x = -1
          # Checks if the robot is not already at the top
          if (posr[1] > 0):
            y = -1
          self.grid.move_by(self, (x, y))


#####*Definimos la clase StackingModel donde se ejecutara toda la secuencia del modelo*
######**Parámetros**
######Tomamos los valores iniciales dados en los parámetros, calculamos la densidad de cajas que habra por espacios en la cuadrícula, una variable para definir el limite de cajas que pueden ser acomodadas como stock en una misma coordenada, inicializamos una variable para contabilizar los movimientos de los agentes, y finalmente una variable para llevar el número de cajas en el stock activo y otra para conocer el número de stocks completados
######**Creación de los Agentes y la Cuadrícula**
######Se inicializan los agentes
######**Update**
######Se realiza una iteración para encontrar coincidencias en las coordenadas de ambas categorías de agentes (StockBot y Cajas) en el actual paso, si existe una coincidencia entonces se cambia el color del StockBot y el estado de los agentes para moverse hacia el StockSpot actual (la pila de cajas donde se están acomodando).
######**Step**
######Ejecuta la función de buscar una nueva celda para realizar el movimiento de los StockBots
######También aumenta el contador de movimientos realizados, la cantidad de cajas apiladas en este momento en el StockSpot activo y una variable que contabiliza los StockSpots (pilas de cajas) completados

In [5]:
class StackingModel(ap.Model):

    def setup(self):

      # Parameters
      h = self.p.height
      w = self.p.width
      d = self.d = int(self.p.density * (w * h))
      n = self.n = self.p.n_agents

      self.num_moves = 0
      self.STOCK_LIMIT = 5
      self.stockFive = 0
      self.stockSpot = 0

      # Create grid and agents
      self.grid = ap.Grid(self, (w, h), track_empty=True, check_border=True)
      self.robotAgents = ap.AgentList(self, n, StackBot)
      self.cellAgents = ap.AgentList(self, d, Cell)
      self.grid.add_agents(self.cellAgents, empty=True, random=True)
      self.grid.add_agents(self.robotAgents, positions = None, empty=True, random=True)
    

    def update(self):
      # Move unhappy people to new location
      self.allRobots = self.robotAgents.select(self.robotAgents.id > 0)
      self.allCells = self.cellAgents.select(self.cellAgents.typeColor == 1)

    def findNearestBox(self, robot, position):
      for neighbor in self.grid.neighbors(robot):
        positionNeighbor = self.grid.positions[neighbor]

        if (neighbor.type == "Cell" and neighbor.stacked == False):
          return neighbor


    def step(self):
      for robot in self.allRobots:
        posRobot = self.grid.positions[robot]
        
        # If the robot is exploring
        if(robot.status == 0):
          nearBox = self.findNearestBox(robot, posRobot)

          if (nearBox != None):
            # Defines that the agents are loading a box
            robot.status = 1
            nearBox.status = 1

            # Defines the box that the robot is loading
            robot.box = nearBox
            
            # Color xd
            nearBox.typeColor = 2

            # Defines that the box is being stacked
            nearBox.stacked = True
            self.d -= 1
        
        # If the robot has a box and it found the goal
        elif posRobot == (0, self.stockSpot) and robot.status == 1:
          # The stock reaches the limit
          if self.stockFive == self.STOCK_LIMIT:
            self.stockFive = 0
            self.stockSpot += 1
            # print("Espacio lleno, redirigiendo a otro punto")

          # The stock is available
          else:
            self.stockFive += 1
            # print("Van ", self.stockFive, " cajas en la posición ", posRobot)
            robot.status = 0
            self.grid.move_to(robot.box, posRobot)
            robot.box = None
        
        robot.find_new_cell(self.stockSpot, posRobot, self.num_moves)
        if(robot.box):
          robot.box.find_new_cell(posRobot)

      self.num_moves += 1

####*Definimos los parámetros especificados para el ejercicio del escenario*

In [6]:
parameters = {
    'n_agents': 5, # Number of StockBots
    'density': 0.3, # Density of boxes
    'height': 10,
    'width': 10,
    'steps': 100  # Maximum number of steps
    }

####*Realizamos la gráficación y animación del modelo, así como el reporte de estado en cada momento de la iteración*

In [7]:
def animation_plot(model, ax):
    gridPosition = model.grid.attr_grid('typeColor')
    color_dict = {0:'#687068', 1:'#f0610a', 2:'#c9997f', 3:'#2be809', 4:'#ffffff', None:'#ffffff'}
    ap.gridplot(gridPosition, ax=ax, color_dict=color_dict, convert=True)
    total = model.p.height * model.p.width
    ax.set_title(f"System to organize and stack boxes \n Time-step: {model.t}, # of Moves: {model.num_moves}, Boxes left: {model.d}")

fig, ax = plt.subplots()
model = StackingModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())